Residual Connection(잔차 연결)
  - 네트웍이 깊어지면 Gradient Vanishing(기울기 소멸문제) 해결
  - 성능향상

In [1]:
import tensorflow as tf
from tensorflow.keras import layers

잔차블럭

In [14]:
from keras.api._v2.keras import activations
inputs = layers.Input(shape=(32,32,3))
x = layers.Conv2D(32,3,activation='relu')(inputs)
residual = x
x = layers.Conv2D(64,3,activation='relu',padding='same')(x)
residual = layers.Conv2D(64,1)(residual)
x = layers.add([x,residual])
x

<KerasTensor: shape=(None, 30, 30, 64) dtype=float32 (created by layer 'add_5')>

최대폴링 가진 잔차블럭

In [13]:
inputs = layers.Input(shape=(32,32,3))
x = layers.Conv2D(32,3,activation='relu')(inputs)
residual = x
x = layers.Conv2D(64,3,activation='relu',padding='same')(x)
x = layers.MaxPooling2D(padding='same')(x)
residual = layers.Conv2D(64,1,strides=2)(residual)
x = layers.add([x,residual])
x

<KerasTensor: shape=(None, 15, 15, 64) dtype=float32 (created by layer 'add_4')>

In [16]:
inputs = layers.Input(shape=(32,32,3))
x = layers.Rescaling(1./255)(inputs)
def residual_block(x, filter, pooling=False):
  residual = x
  x = layers.Conv2D(filter,3,activation='relu',padding='same')(x)
  x = layers.Conv2D(filter,3,activation='relu',padding='same')(x)
  if pooling:
    x = layers.MaxPooling2D(2, padding='same')(x)
    residual = layers.Conv2D(filter,1,strides=2)(residual)
  elif filter != residual.shape[-1]:
    residual = layers.Conv2D(filter,1)(residual)
  x = layers.add([x,residual])
  return x

x = residual_block(x,filter=32,pooling=True)
x = residual_block(x,filter=64,pooling=True)
x = residual_block(x,filter=128,pooling=False)

x = layers.GlobalAveragePooling2D()(x)  # Flatten을 안해도.... 일차원 벡터로 변경해준다.
outputs = layers.Dense(1,activation='sigmoid')(x)
model = tf.keras.Model(inputs,outputs)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 rescaling_1 (Rescaling)        (None, 32, 32, 3)    0           ['input_10[0][0]']               
                                                                                                  
 conv2d_23 (Conv2D)             (None, 32, 32, 32)   896         ['rescaling_1[0][0]']            
                                                                                                  
 conv2d_24 (Conv2D)             (None, 32, 32, 32)   9248        ['conv2d_23[0][0]']              
                                                                                              

real 이미지
  - 폴더별 클래스를 만들고 해당 크래스폴더에 이미지 데이터
  - 상위 폴더를 image_dataset_from_directory를 적용하면
    - 이미지셋과 라벨셋
  - 이미지 agumentation 증폭
  - 배치정규화(과도한 규제 및 dropout을 피할수 있음)
